In [ ]:
from random import random
import numpy as np
from itertools import combinations
import math
import random

In [ ]:
# increment the degree of each vertex on graph g by 1
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g

In [ ]:
# decrement the degree of each vertex on graph g to 0
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g


In [ ]:
### create a graph with planted clique of size = clique_size

# g: graph (represented as a tensor)
# num_vertices: number of vertices on the graph (dimension, or rank of the tensor) 
# pr: probability of creating edges
# clique_size: clique size

def generate_graph(num_vertices, pr, clique_size):
    def plant_clique():
        for i in range(clique_size):
            for j in range(i + 1, clique_size):
                for l in range(j + 1, clique_size):
                    a = clique_vertices[i]
                    # print(a)
                    vec[a] += 1
                    b = clique_vertices[j]
                    # print(b)
                    vec[b] += 1
                    c = clique_vertices[l]
                    # print(c)
                    vec[c] += 1
                    add_all(a, b, c, g)

    g = np.array([np.array([np.array([0 for i in range(0, num_vertices)]) for i in range(num_vertices)]) for i in
                  range(num_vertices)])
    vec = np.array([0 for i in range(0, num_vertices)])

    # Set edges
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                a = np.random.uniform(0, 1, 1)
                # every edge is included independently with probability 1/2
                if a < pr:
                    vec[i] += 1
                    vec[j] += 1
                    vec[k] += 1
                    add_all(i, j, k, g)

    # clique_vertices = random.sample(range(num_vertices), clique_size) # is is wrong??; would violate the clique size constraint
    
    clique_vertices = np.random.choice(range(num_vertices), clique_size, replace=False)

    plant_clique()

    return g, vec, clique_vertices


In [ ]:
# calculate the total number of edges on a graph with num_vertices vertices
def find_num_edges(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [ ]:
# check if the graph is a clique
def is_clique(g, vec, num_vertices):
    # active_count: number of vertices that are associated with at least 1 edge
    active_count = np.count_nonzero(vec)
    # number of edges
    edge_sum = find_num_edges(g, num_vertices)
    if edge_sum == math.comb(active_count, 3):
        return True
    return False

In [ ]:
# remove edge
def remove_edges(g, num_vertices, vec, curr):
    vec[curr] = 0
    graph_copy = g.copy()
    for j in range(0, num_vertices):
        for k in range(0, num_vertices):
            if graph_copy[curr, j, k] == 1:
                vec[j] -= 1
                vec[k] -= 1
                remove_all(curr, j, k, graph_copy)
    return graph_copy, vec

### Generate a graph

In [ ]:
# driver program

n = 1600 # number of vertices
p = 0.5 # probablity of edge being included
k = 33 # clique size

# generate graph
res = generate_graph(n, p, k)

# G graph with planted clique
# V: vector storing the number of edges associated with each vertice in the graph
# clique_vertices: the set of clique vertices
G, V, clique_vertices = res[0],  res[1], res[-1]

# print(f"graph: {G}\n")
G_0 = G.copy()
print(f"edge-occurence vector: {V}\n")
print(f"clique vertices: {clique_vertices}\n")



### Removal Phase

In [ ]:
iter = 0
removed = []

clique = is_clique(G, V, n)
while not clique:
    
    iter += 1

    curr = -1
    idx_sorted = np.argsort(V)
    
    print(f"i = {iter}\nindex sorted: {idx_sorted}")
    print(f"number of edges associated with each vertex: {V}")
    # print(f'current set of clique vertices: {clique_vertices}')

    for idx in range(n):
        if V[idx_sorted[idx]] != 0:
            curr = idx_sorted[idx]
            removed.append(curr)
            break
    print(f"vertex removed: {curr} number of edges: {V[curr]}\n")
    A = remove_edges(G, n, V, curr)
    # print(A[1])
    G = A[0]

    clique = is_clique(G, V, n)
print("clique!!!")

In [ ]:
iter

In [ ]:
sorted(removed)

### Inclusion Phase

In [ ]:
full_vertices = np.arange(n)
full_vertices

In [ ]:
included = np.setdiff1d(full_vertices, removed)
included

In [ ]:
def inclusion_phase(removed, in_set, G_0):
    def connected(idx, in_set):
        for j in range(len(in_set)):
                for k in range(j+1, len(in_set)):
                    if G_0[idx, in_set[j], in_set[k]] != 1:
                        print(f" vertex {idx} is not connected to all clique vertices")  
                        return False
        return True
                        
    for idx in removed:
        if connected(idx, in_set):
            print(f"add {idx} to clique!")
            in_set = np.append(in_set, idx)

    return in_set
    

In [ ]:
res = inclusion_phase(removed, included, G_0)
res